In [86]:
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import plotly.express as px
import pandas as pd
import folium
import streamlit as st
import folium
from streamlit_folium import st_folium
from shapely.geometry import box

In [87]:
pays_de_la_loire = gpd.read_file('Pays_de_la_Loire.json')
pays_de_la_loire_enjeu_2_3 = pays_de_la_loire[(pays_de_la_loire['enjeu'] == 2) | (pays_de_la_loire['enjeu'] == 3)]
# pays_de_la_loire_enjeu_2_3 = pays_de_la_loire.copy()

latitude = 47.98199   # Replace with your desired latitude
longitude = 0.1133009

# Create a base map
m = folium.Map(location=[latitude, longitude], zoom_start=2)  # Adjust latitude and longitude to your region

pays_de_la_loire_enjeu_2_3['region'] = 'PAYS DE LA LOIRE'
columns_to_keep = ['region', 'enjeu', 'geometry']

pays_de_la_loire_enjeu_2_3 = pays_de_la_loire_enjeu_2_3[columns_to_keep]
pays_de_la_loire_enjeu_2_3





c:\Users\j3r3m\miniconda3\envs\final_project\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,region,enjeu,geometry
0,PAYS DE LA LOIRE,2,"MULTIPOLYGON (((416294 6634866, 416324 6634876..."
3,PAYS DE LA LOIRE,3,"MULTIPOLYGON (((543406.563 6786093.228, 543367..."


In [89]:
file_paths = [
    # "aura_enjeu_0.gpkg",
    # "aura_enjeu_1.gpkg",
    "aura_enjeu_2.gpkg",
    "aura_enjeu_3.gpkg",
]
# Initialize an empty list to store each GeoDataFrame
gdfs = []

# Loop through each file path and read the GeoPackage file
for file_path in file_paths:
    # Read the GeoPackage into a GeoDataFrame
    gdf = gpd.read_file(file_path)
    
    # Append the GeoDataFrame to the list
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
aura_enjeu_2_3 = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
aura_enjeu_2_3.rename(columns={'code_enjeu': 'enjeu'}, inplace=True)
aura_enjeu_2_3 = aura_enjeu_2_3[columns_to_keep]

aura_enjeu_2_3


,region,enjeu,geometry
0,AUVERGNE RHONE ALPES,2,"MULTIPOLYGON (((782722.299 6375074.051, 782715..."
1,AUVERGNE RHONE ALPES,3,"MULTIPOLYGON (((786495.409 6359441.378, 786480..."


In [90]:
df_final = pd.concat([pays_de_la_loire_enjeu_2_3, aura_enjeu_2_3], ignore_index=True)
df_final['enjeu'] = df_final['enjeu'].astype(int)
df_final

,region,enjeu,geometry
0,PAYS DE LA LOIRE,2,"MULTIPOLYGON (((416294 6634866, 416324 6634876..."
1,PAYS DE LA LOIRE,3,"MULTIPOLYGON (((543406.563 6786093.228, 543367..."
2,AUVERGNE RHONE ALPES,2,"MULTIPOLYGON (((782722.299 6375074.051, 782715..."
3,AUVERGNE RHONE ALPES,3,"MULTIPOLYGON (((786495.409 6359441.378, 786480..."


In [91]:
# Define the style function to apply different colors based on the 'enjeu' property
def style_function(feature):
    enjeu = feature['properties'].get('enjeu', 0)
    color_map = {
        0: 'white',    # Color for 'enjeu' = 0
        1: 'blue',    # Color for 'enjeu' = 1
        2: 'orange',   # Color for 'enjeu' = 2
        3: 'red',     # Color for 'enjeu' = 3
        'default': 'grey'  # Default color
    }
    return {
        'fillColor': color_map.get(enjeu, color_map['default']),
        'color': 'white',         # Border color
        'weight': 0,            # Border weight
        'fillOpacity': 0.5        # Fill opacity
    }

# Convert the GeoDataFrame to GeoJSON and add it to the map
folium.GeoJson(
    df_final,
    style_function=style_function,
    name='Zones of Interest'
).add_to(m)

folium.TileLayer('cartodbpositron').add_to(m)



In [92]:
# Add layer control if you have multiple layers
folium.LayerControl().add_to(m)

# Create the legend
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 150px; height: auto; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color: white;
                ">
    &nbsp; <b>Zone Legend</b><br>
    &nbsp; <i style="background: white; width: 24px; height: 24px; display: inline-block; border-radius: 50%;"></i> Zone redhibitoire<br>
    &nbsp; <i style="background: blue; width: 24px; height: 24px; display: inline-block; border-radius: 50%;"></i> Zone non potentiellement favorable<br>
    &nbsp; <i style="background: orange; width: 24px; height: 24px; display: inline-block; border-radius: 50%;"></i> Zone potentiellement favorable<br>
    &nbsp; <i style="background: red; width: 24px; height: 24px; display: inline-block; border-radius: 50%;"></i> Zone potentiellement favorable<br>
    </div>
'''

# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save('map_test.html')

In [93]:


# # Display the Folium map in the Streamlit app
# st.title('Folium Map in Streamlit')
# st_folium(m, width=700, height=500)


In [ ]:
# import fiona

# # List all layers in the GeoPackage
# layers = fiona.listlayers("sre_cartepotentieleolien_2022_r84_enjeu_1.gpkg")
# print(layers)
